# About

In this notebook, we will motivate RAG.

We will see why we can't use Large Language Model directly.



# Imports

In [16]:
from dotenv import load_dotenv
import rich
import logging
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI
from huggingface_hub import InferenceClient
import os

In [5]:
#logging.basicConfig(level=logging.DEBUG)


In [6]:
load_dotenv(dotenv_path="../env")

True

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo",temperature=0)


# Problems

There are atleast two problems with using Large Language Model 

- Knoweldge Cutoff
- Hallucination

## Knoweldge Cutoff

In [7]:
llm = OpenAI(model="gpt-4o-mini",temperature=0)


In [12]:
def get_response(query:str):
    messages = [
        ChatMessage(role="user", content=query),
    ]
    
    resp = llm.chat(messages)

    return resp

In [13]:
content="Who won the last US election"


In [10]:
resp = get_response(content)

In [11]:
rich.print(resp)

ChatResponse(
    message=ChatMessage(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='As of my last update in October 2023, Joe Biden won the last U.S. presidential election, which 
took place on November 3, 2020. He officially took office on January 20, 2021. If you are referring to a different 
election, such as midterm elections or local elections, please specify, and I can provide more information.',
        additional_kwargs={}
    ),
    raw=ChatCompletion(
        id='chatcmpl-AOCa83fVmE52cD0XMeFWg5QjuKIJv',
        choices=[
            Choice(
                finish_reason='stop',
                index=0,
                logprobs=None,
                message=ChatCompletionMessage(
                    content='As of my last update in October 2023, Joe Biden won the last U.S. presidential 
election, which took place on November 3, 2020. He officially took office on January 20, 2021. If you are referring
to a different election, such as midterm elections or local elections, please specify, and I can provide more 
information.',
                    refusal=None,
                    role='assistant',
                    audio=None,
                    function_call=None,
                    tool_calls=None
                )
            )
        ],
        created=1730331600,
        model='gpt-4o-mini-2024-07-18',
        object='chat.completion',
        service_tier=None,
        system_fingerprint='fp_0ba0d124f1',
        usage=CompletionUsage(
            completion_tokens=75,
            prompt_tokens=13,
            total_tokens=88,
            completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0),
            prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)
        )
    ),
    delta=None,
    logprobs=None,
    additional_kwargs={'prompt_tokens': 13, 'completion_tokens': 75, 'total_tokens': 88}
)

note the model is aliased to `model='gpt-4o-mini-2024-07-18'`.

So, the model's training data wont have any information after then.

In [24]:
!pwd

/home/jupyter/pydata_rag_video/notebooks


## Hallucination

![Snapshot of Hallucination](../images/llama_2__hallucination.png)


In the above video, we ask about non existant Llama model.

We also give it a link to a study music youtube video.

Note: we are also using an "older" version of Llama model to make the inference

In [27]:
client = InferenceClient(api_key=os.environ['HF_API_KEY'])

In [28]:
message = f"""
What did Andrej Karpathy say about Meta's Llama 5.9 in the below youtube talk

https://www.youtube.com/watch?v=n61ULEU7CO0&ab_channel=LofiGirl

"""

In [29]:
messages = [
    { "role": "user", "content": message },
]

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf", 
    messages=messages, 
    temperature=0,
)

In [30]:
rich.print(response)

ChatCompletionOutput(
    choices=[
        ChatCompletionOutputComplete(
            finish_reason='length',
            index=0,
            message=ChatCompletionOutputMessage(
                role='assistant',
                content="  Andrej Karpathy, a well-known computer scientist and researcher, did not make any 
comments about Meta's Llama 5.9 in the YouTube talk you provided. The talk is actually about the use of language 
models in music generation, and Karpathy's comments are focused on the topic of language models and their 
applications in various fields.\n\nTherefore, I cannot provide any information about what Karpathy said about 
Meta's Llama",
                tool_calls=None
            ),
            logprobs=None
        )
    ],
    created=1730331898,
    id='',
    model='meta-llama/Llama-2-7b-chat-hf',
    system_fingerprint='2.3.1-dev0-sha-169178b',
    usage=ChatCompletionOutputUsage(completion_tokens=100, prompt_tokens=67, total_tokens=167)
)

In [31]:
rich.print(response.choices[0].message.content)

Andrej Karpathy, a well-known computer scientist and researcher, did not make any comments about Meta's Llama 5.9
in the YouTube talk you provided. The talk is actually about the use of language models in music generation, and 
Karpathy's comments are focused on the topic of language models and their applications in various fields.

Therefore, I cannot provide any information about what Karpathy said about Meta's Llama